In [1]:
from glob import glob

In [2]:
import sys
sys.path.append('../src/')

from core import CVMEME

In [3]:
cvmeme = CVMEME()

In [4]:
cv_path = glob('../data/cv/*')[0]

In [5]:
cv_path

'../data/cv/Edwin_Paco_Software_Developer.pdf'

In [6]:
cv, cv_processed = cvmeme.process_cv(cv_path)

In [7]:
cv_scores = cvmeme.get_scores(cv_processed)

In [8]:
cv_scores

['пух_смокинге_шаблон', 'сверхразум', 'da_poebat_mne_gosling']

In [9]:
meme = cvmeme.rerank_memes(cv, cv_scores)

In [10]:
meme

'пух_смокинге_шаблон'

In [11]:
img = cvmeme.get_meme_img_path(meme)

In [12]:
img

'../data/meme/img/пух_смокинге_шаблон.jpg'

In [13]:
import telegram
telegram.__version__

'21.4'

In [4]:
cvmeme.get_other_respond("привет")

In [5]:
from random import random, choices

random()

0.3284822757205327

In [9]:
choices([1, 2, 3])

[1]